In [10]:
import sys
sys.path.insert(0,"/home/jovyan/fairing")
print(sys.path)

['/home/jovyan/fairing', '/home/jovyan/fairing', '/opt/conda/lib/python36.zip', '/opt/conda/lib/python3.6', '/opt/conda/lib/python3.6/lib-dynload', '', '/opt/conda/lib/python3.6/site-packages', '/opt/conda/lib/python3.6/site-packages/IPython/extensions', '/home/jovyan/.ipython']


In [11]:
import sys
print(sys.version)

3.6.7 | packaged by conda-forge | (default, Feb 28 2019, 09:07:38) 
[GCC 7.3.0]


In [12]:
import os
import fairing

In [13]:
import logging
logger = logging.getLogger()

logging.basicConfig(level=logging.DEBUG)
logging.debug('This will get logged')


In [14]:
# Setting up google container repositories (GCR) for storing output containers
# You can use any docker container registry istead of GCR
#GCP_PROJECT = fairing.cloud.gcp.guess_project_name()
#DOCKER_REGISTRY = 'mtr.external.otc.telekomcloud.com/rainer_englisch'
#DOCKER_REGISTRY = 'docker.io/m1st3rb3an'
#DOCKER_REGISTRY = 'registry.hub.docker.com/m1st3rb3an'
DOCKER_REGISTRY = 'index.docker.io/m1st3rb3an'

BASE_IMAGE ='tensorflow/tensorflow:nightly-py3-jupyter'
#fairing.config.set_builder('append',base_image='tensorflow/tensorflow:latest-py3', registry=DOCKER_REGISTRY, push=True)
# Rainer: need to use latest base_image because of newer python. The older version in base image is not compatible with one in notebook in regards to serialization
#fairing.config.set_builder('append',base_image='tensorflow/tensorflow:devel-py3', registry=DOCKER_REGISTRY, push=True)
#fairing.config.set_builder('append',base_image='tensorflow/tensorflow:1.13.1-py3', registry=DOCKER_REGISTRY, push=True)
fairing.config.set_builder('append',base_image=BASE_IMAGE, registry=DOCKER_REGISTRY, push=True)
fairing.config.set_deployer('job')

In [15]:
"""Trains and Evaluates the MNIST network using a feed dictionary."""
import os

from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist


#import fairing
#fairing.config.set_builder('docker', registry='<your-registry-name>',    base_image='tensorflow/tensorflow:1.13.1-py3')
#fairing.config.run()

INPUT_DATA_DIR = '/tmp/tensorflow/mnist/input_data/'
MAX_STEPS = 2000
BATCH_SIZE = 100
LEARNING_RATE = 0.3
HIDDEN_1 = 128
HIDDEN_2 = 32

# HACK: Ideally we would want to have a unique subpath for each instance of the job, but since we can't
# we are instead appending HOSTNAME to the logdir
LOG_DIR = os.path.join(os.getenv('TEST_TMPDIR', '/tmp'),
                       'tensorflow/mnist/logs/fully_connected_feed/', os.getenv('HOSTNAME', ''))
MODEL_DIR = os.path.join(LOG_DIR, 'model.ckpt')

def train():
    
    #logger.info("Importing libs")
    #import tensorflow as tf
    #from tensorflow.examples.tutorials.mnist import input_data
    #from tensorflow.examples.tutorials.mnist import mnist
    logger.info("Start training")
    data_sets = input_data.read_data_sets(INPUT_DATA_DIR)
    images_placeholder = tf.placeholder(tf.float32, shape=(BATCH_SIZE, mnist.IMAGE_PIXELS))
    labels_placeholder = tf.placeholder(tf.int32, shape=(BATCH_SIZE))

    logits = mnist.inference(images_placeholder, HIDDEN_1, HIDDEN_2)
    
    loss = mnist.loss(logits, labels_placeholder)
    train_op = mnist.training(loss, LEARNING_RATE)
    summary = tf.summary.merge_all()
    init = tf.global_variables_initializer()
    sess = tf.Session()
    summary_writer = tf.summary.FileWriter(LOG_DIR, sess.graph)
    sess.run(init)
    
    data_set = data_sets.train
    logger.info("Start training")
    for step in xrange(MAX_STEPS):
        images_feed, labels_feed = data_set.next_batch(BATCH_SIZE, False)
        feed_dict = {
            images_placeholder: images_feed,
            labels_placeholder: labels_feed,
        }

        _, loss_value = sess.run([train_op, loss],
                                 feed_dict=feed_dict)
        if step % 100 == 0:
            print("At step {}, loss = {}".format(step, loss_value))
            summary_str = sess.run(summary, feed_dict=feed_dict)
            summary_writer.add_summary(summary_str, step)
            summary_writer.flush()
    logger.info("Save my model")
    saver = tf.train.Saver()
    saver.save(sess, 'my_mnist_model')
    logger.info("End training")

# Local training
train()

In [16]:
#fairing.config.run()

In [17]:
remote_train = fairing.config.fn(train)
remote_train()
#fairing.config.run()

Using preprocessor: <fairing.preprocessors.function.FunctionPreProcessor object at 0x7fad2df564a8>
Using builder: <fairing.builders.append.append.AppendBuilder object at 0x7fad600310f0>
Building image using Append builder...
Creating docker context: /tmp/fairing_context_h6hyrhzq
/home/jovyan/fairing/fairing/__init__.py already exists in Fairing context, skipping...
Loading Docker credentials for repository 'tensorflow/tensorflow:nightly-py3-jupyter'
Image successfully built in 1.3086363889997301s.
Pushing image index.docker.io/m1st3rb3an/fairing-job:22FAAB03...
Loading Docker credentials for repository 'index.docker.io/m1st3rb3an/fairing-job:22FAAB03'
Uploading index.docker.io/m1st3rb3an/fairing-job:22FAAB03
Layer sha256:05731e63f21105725a5c062a725b33a54ad8c697f9c810870c6aa3e3cd9fb6a2 exists, skipping
Layer sha256:e0b421c621114fc389e8dfa0414c80c2f793e7dc2efe2d449830890bb21afd83 exists, skipping
Layer sha256:30c30f323f555817df41239125e472e408a81167b64a7039de0054ec28e33570 exists, skippi

I0603 20:26:32.434225 139934835124032 <ipython-input-15-d09545b67cbb>:33] Start training
W0603 20:26:32.434432 139934835124032 deprecation.py:323] From <ipython-input-15-d09545b67cbb>:34: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0603 20:26:32.434516 139934835124032 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0603 20:26:32.434799 139934835124032 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (

Cleaning up job fairing-job-ftz62...


In [18]:

from fairing import TrainJob
from fairing.backends import KubeflowBackend, KubernetesBackend
train_job = TrainJob(train, BASE_IMAGE,# input_files=['ames_dataset/train.csv', "requirements.txt"],
                     docker_registry=DOCKER_REGISTRY, backend=KubernetesBackend())
train_job.submit()

Using preprocessor: <class 'fairing.preprocessors.function.FunctionPreProcessor'>
Using docker registry: index.docker.io/m1st3rb3an
Using builder: <class 'fairing.builders.append.append.AppendBuilder'>
Building the docker image.
Building image using Append builder...
Creating docker context: /tmp/fairing_context_6y7sy8vt
/home/jovyan/fairing/fairing/__init__.py already exists in Fairing context, skipping...
Loading Docker credentials for repository 'tensorflow/tensorflow:nightly-py3-jupyter'
Image successfully built in 1.3038988529997368s.
Pushing image index.docker.io/m1st3rb3an/fairing-job:2E2F0406...
Loading Docker credentials for repository 'index.docker.io/m1st3rb3an/fairing-job:2E2F0406'
Uploading index.docker.io/m1st3rb3an/fairing-job:2E2F0406
Layer sha256:05731e63f21105725a5c062a725b33a54ad8c697f9c810870c6aa3e3cd9fb6a2 exists, skipping
Layer sha256:868fe117e0a60fd3c6c0561ee074e68b8f53339cfb2f56ac511f6880a6c9287f exists, skipping
Layer sha256:526bb5d0ae6c26ed478aa60ae185d5652e5c

I0603 20:26:52.583691 139972787337024 <ipython-input-15-d09545b67cbb>:33] Start training
W0603 20:26:52.583896 139972787337024 deprecation.py:323] From <ipython-input-15-d09545b67cbb>:34: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0603 20:26:52.583984 139972787337024 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0603 20:26:52.584267 139972787337024 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (

Cleaning up job fairing-job-n5lsx...


'fairing-job-n5lsx'